# Transient diffusion equation

### PDE and boundary conditions
The transient diffusion equation reads

 $$\alpha\frac{\partial c}{\partial t}+\nabla.\left(-D\nabla c\right)=0,$$

 where $c$ is the independent variable (concentration, temperature, etc)
 , $D$ is the diffusion coefficient, and $\alpha$ is a constant.
 
 
 This is an example program for the python extension of the MATLAB toolbox FVTool
 
 Written by Ali A. Eftekhari
 Last checked: June 2021
 
 Ported to python by Gavin M. Weir June, 2023


In [ ]:
import numpy as np
import pyfvtool as pyfvm
from pyfvtool import *

In [ ]:
# Define the domain and create a mesh structure
L = float(10)   # domain length
Nx = int(20)        # number of cells
Ntetta = int(21)    # number of polar angle cells

# Generate the mesh 
m = createMeshRadial2D(Nx, Ntetta, L, 2.0*np.pi)   # Nx: int, Ny: int, Lx: float, Ly: float (Nr, Ntetta, Lr, Tetta)

In [ ]:
# Create the boundary condition structure
BC = createBC(m)  # all Neumann boundary condition structure

# Add a Ghost cell
m.cellcenters.x = m.cellcenters.x+1
m.facecenters.x = m.facecenters.x+1

# left boundary
BC.left.a[:20:3], BC.left.b[:20:3], BC.left.c[:20:3] = 0.0, 1.0, 10.0 

# right boundary
BC.right.a[:], BC.right.b[:], BC.right.c[:] = 0.0, 1.0, 0.0
# BC.right.a[:int(Ntetta//2)], BC.right.b[:int(Ntetta//2)], BC.right.c[:int(Ntetta//2)] = 0.0, 1.0, 3.0

# Top boundary
BC.top.periodic = True
# BC.top.a[:], BC.top.b[:], BC.top.c[:] = 0.0, 1.0, 0.0

# Bottom boundary  -  if unspecified, this defaults to homogeneous Neumann
# BC.bottom.a[:], BC.bottom.b[:], BC.bottom.c[:] = 0, 1, 0

In [ ]:
# define the transfer coefficients

# Diffusion term
D_val = 0.01
D = createCellVariable(m, D_val)

# Transient term
alfa = createCellVariable(m, 1)

# Advection term
u_val=1
u = createFaceVariable(m, [u_val, 0]);

In [ ]:
# define initial values
c_init = 0.1
c_old = createCellVariable(m, c_init,BC)  # initial values
 
# assign the old value of the cells to the current values
c = c_old

In [ ]:
# loop
dt = 0.5  # time step
final_t = 7

# Interpolate diffusion cell values to face values
Dave = harmonicMean(D)

# Create the coefficient matrices
Mdiff = diffusionTerm(Dave)
Mbc, RHSbc = pyfvm.boundary.boundaryCondition(BC)

# Pick a flux-limiter
FL = fluxLimiter('Superbee')

# Define the convection term
Mconv = convectionTerm(u)

ci = []
for tt in np.arange(start=dt, stop=final_t, step=dt):  # t=dt:dt:final_t
    
    # Transient term
    M_trans, RHS_trans = transientTerm(c, dt, alfa)
    
    # Form the linear system of equations
    M = M_trans + Mconv - Mdiff + Mbc
    RHS = RHS_trans + RHSbc
    
    # Solve the system of equations
    c = solvePDE(m,M, RHS)
    
    # Update the stored solutions
    c_old = c.value
    ci.append(c)

In [ ]:
# Plotting and visualization 
from matplotlib.animation import FuncAnimation

hfig1, ax1 = plt.subplots()
ax1.plot(x, ci[0].value[1:Nx+1], 'r-', label='initial value')
ax1.plot(x, ci[-1].value[1:Nx+1], 'b-', label='final value')

ax1.set_xlim((0, L))
ax1.set_ylim((-1.25, 1.25))

ax1.set_xlabel('x')
ax1.set_ylabel('c')
ax1.set_title('2D Transient convection-diffusion')
ax1.legend(fontsize=8)

# ========= #

# create a figure to handle the solution animation
#   pre-generate the handles to the axes and the line
hfig2 = plt.figure()
ax2 = plt.axes(xlim=(0, L), ylim=(-1.25, 1.25))
line, = ax2.plot([], [], 'r-', lw=3, label='Numerical solution')
ax2.set_xlabel('x')
ax2.set_ylabel('c')
ax2.set_title('2D transient convective-diffusion: Constant coeff.')
ax2.legend(fontsize=8)

def init():
    line.set_data([], [])
    return line

def animate(ii):
    global x
    global ci
    global Nx
    line.set_data(x, ci[ii].value[1:Nx+1])    
    return line

anim = FuncAnimation(hfig2, animate, init_func=init,
            frames=400, interval=20, blit=True)